In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import VGG19
from keras.applications import VGG19
from keras.models import Sequential
from keras.layers import Flatten, Dense
from tensorflow.keras.layers import Flatten, Dense, Dropout


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Covid19-dataset'
train_dir = '/content/drive/MyDrive/Covid19-dataset/train'
test_dir = '/content/drive/MyDrive/Covid19-dataset/test'

Mounted at /content/drive


In [ ]:
vgg = VGG19(include_top = False , input_shape = (224,224,3),weights='imagenet')
for layer in vgg.layers :
    layer.trainable = False

ES = tf.keras.callbacks.EarlyStopping(
    patience = 10,
    min_delta = 0.001,
    verbose = 0
)
vgg = tf.keras.Sequential([
    layers.RandomFlip(mode='horizontal'),
    layers.RandomRotation(factor=0.2),
    vgg,
    layers.Flatten(),
    layers.Dense(3, activation='softmax')
])
vgg.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

80134624/80134624 [==============================] - 3s 0us/step


In [ ]:
image_shape = (224, 224)
batch_size = 32

# Data augmentation and preprocessing for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting the training set into 80% training and 20% validation
)

# Preprocessing for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and augment the training set
train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_shape,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'  # Use the subset 'training' for the training set
)

# Load and augment the validation set
val_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_shape,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'  # Use the subset 'validation' for the validation set
)

# Load the test set
test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_shape,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)
# fit the model
training = vgg.fit_generator(
    train_set,
    callbacks=[ES],
    steps_per_epoch = 7 ,
    epochs = 30,
    validation_data = val_set
)

accuracy = training.history['accuracy']
validation_accuracy = training.history['val_accuracy']

final_accuracy = accuracy[-1]
final_validation_accuracy = validation_accuracy[-1]

print("Final Training Accuracy:", final_accuracy)
print("Final Validation Accuracy:", final_validation_accuracy)

Found 201 images belonging to 3 classes.
Found 50 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


<ipython-input-5-5d427a1597a2>:46: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  training = vgg.fit_generator(


Epoch 1/30
7/7 [==============================] - 208s 30s/step - loss: 2.3860 - accuracy: 0.3881 - val_loss: 2.2929 - val_accuracy: 0.4800
Epoch 2/30
7/7 [==============================] - 183s 26s/step - loss: 1.4141 - accuracy: 0.6368 - val_loss: 1.8066 - val_accuracy: 0.4200
Epoch 3/30
7/7 [==============================] - 227s 34s/step - loss: 0.9571 - accuracy: 0.6269 - val_loss: 0.6992 - val_accuracy: 0.6600
Epoch 4/30
7/7 [==============================] - 187s 27s/step - loss: 0.6391 - accuracy: 0.7463 - val_loss: 0.7906 - val_accuracy: 0.6200
Epoch 5/30
7/7 [==============================] - 185s 30s/step - loss: 0.6030 - accuracy: 0.7413 - val_loss: 0.8688 - val_accuracy: 0.6200
Epoch 6/30
7/7 [==============================] - 179s 26s/step - loss: 0.4969 - accuracy: 0.7662 - val_loss: 0.6111 - val_accuracy: 0.7400
Epoch 7/30
7/7 [==============================] - 190s 27s/step - loss: 0.4749 - accuracy: 0.8408 - val_loss: 0.5046 - val_accuracy: 0.8800
Epoch 8/30
7/7 [====

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

test_loss, test_accuracy = vgg.evaluate(test_set)
test_set.reset()
predictions = vgg.predict(test_set)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_set.classes

# Compute precision, recall, and F1 score
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# Print final accuracies and metrics
print("Final Training Accuracy:", training.history['accuracy'][-1])
print("Final Validation Accuracy:", training.history['val_accuracy'][-1])
print("Test Accuracy:", test_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Test Loss:", test_loss)

# Print a detailed classification report
print("\nClassification Report:\n", classification_report(true_classes, predicted_classes))

3/3 [==============================] - 47s 12s/step
Final Training Accuracy: 0.8905472755432129
Final Validation Accuracy: 0.8999999761581421
Test Accuracy: 0.939393937587738
Precision: 0.9418426691153964
Recall: 0.9393939393939394
F1 Score: 0.9392420445052023
Test Loss: 0.1905670017004013

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       0.94      0.85      0.89        20
           2       0.86      0.95      0.90        20

    accuracy                           0.94        66
   macro avg       0.94      0.93      0.93        66
weighted avg       0.94      0.94      0.94        66

